# Importing the necessary APIs

In [1]:
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
hf_api_key = userdata.get('HUGGINGFACEHUB_API_TOKEN')
qdrant_api_key = userdata.get('QDRANT_API_KEY')

# Importing the Necessary Dependencies

In [2]:
!pip install langchain langchain_community langchain_google_genai langchain_groq qdrant-client langchain_huggingface langchain_qdrant neo4j langchain_neo4j langgraph pydantic airtop tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 

In [3]:
!pip install --upgrade langchain langchain_neo4j

# Using the Retriever and Langchain Common Expression Language (LCEL)

In [4]:
from IPython.display import display, Markdown
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
url = "https://fbee74c7-ab4b-4ba0-be6e-85d5ca73c42a.us-west-1-0.aws.cloud.qdrant.io"
client = QdrantClient(url=url, api_key=qdrant_api_key)
collection_name="haystack_warrior"
qdrant_store = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embeddings
)
retriever = qdrant_store.as_retriever()


# Define the prompt template
prompt = ChatPromptTemplate.from_template("""
    Answer the following question on the given context as well as from your
    base cut-off knowledge , while
    answering the queries of the user , Also provide url links to the documentations
    wherever necessary
    The response should be clear,detail,structured and in-depth, provide code
    snippets wherever necessary ,
    <context>
    {context}
    </context>
    Question: {input}""")

llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    api_key=userdata.get("GOOGLE_API_KEY"),
    streaming=True
)

query = "How is langchain different from haystacks , tabulate the key differences, are you using a context? , Also tell me what was the source of the context that made you help this answer in a contextually relevant manner, do you think this RAG approach I used to build you was sufficient or your raw knowledge was okay ? I think I wasted my time then"
retrieval_chain = create_stuff_documents_chain(llm, prompt=prompt)
QA_chain  = create_retrieval_chain(retriever,retrieval_chain)
response = QA_chain.invoke({"input": query} )
display(Markdown(response["answer"]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-e04ba473b412>:23: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  qdrant_store = Qdrant(


Okay, let's break down the differences between Haystack and LangChain, and address your questions about context and my knowledge.

**Haystack vs. LangChain: Key Differences**

It's important to note that both Haystack and LangChain are frameworks designed to work with Large Language Models (LLMs), but they approach the problem with different philosophies and strengths. Here's a table summarizing their key differences:

| Feature             | Haystack                                                                                                                                                                                                                             | LangChain                                                                                                                                                                                                                                                           |
|----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Primary Focus**    | Building end-to-end search and question answering systems, with a strong emphasis on document retrieval and processing.                                                                                                                 | Developing a wide variety of applications powered by LLMs, including chatbots, text summarization, data analysis, and more.                                                                                                                                    |
| **Architecture**     | Uses a pipeline-based architecture where components are connected sequentially to process data.  Focuses on modular components that handle specific tasks like document loading, embedding, retrieval, and generation.                   | Emphasizes modularity and flexibility, with a focus on chains of LLM calls. Provides building blocks for creating custom agents, tools, and memory management.                                                                                                          |
| **Document Handling** | Strong focus on document indexing, retrieval, and processing. Offers various document stores, embedders, and retrievers. Provides robust tools for working with structured and unstructured data.                                       | Supports document handling but is less specialized in this area compared to Haystack.  Offers tools for loading and splitting documents but might require more manual configuration for complex document processing scenarios.                                 |
| **Integration**      | Rich ecosystem of integrations for various document stores, embedding models, and other services. Tightly integrates with popular tools for search and retrieval.                                                                     |  Broad ecosystem of integrations with various LLMs, data sources, and tools. Offers more variety in terms of LLM providers and external tool usage.                                                                                                              |
| **Customization**    | Components are customizable, but the overall pipeline structure is more defined. Offers mechanisms for creating custom components, but might require a deeper understanding of the framework.                                          | Highly customizable and flexible.  Allows for creation of custom chains, agents, and tools.  Suited for developers who need fine-grained control over the LLM application's behavior.                                                                       |
| **Complexity**       | Easier to get started with for search and QA tasks, with a more opinionated approach. May require more effort to deviate from the standard pipeline structure.                                                                         |  Offers more flexibility, but can be more complex to learn and configure.  Requires more understanding of LLM concepts and best practices to build robust applications.                                                                                             |
| **Use Cases**        | Ideal for building search engines, question-answering systems, knowledge bases, and other applications that heavily rely on document retrieval.                                                                                           |  Suitable for a wider range of applications, including chatbots, text generation, data analysis, and more. Offers more flexibility for creating custom agents and tools.                                                                                             |
| **Community & Maturity** |  Haystack is gaining popularity, but LangChain has a larger and more established community.                                                                                                                                    |  LangChain is very popular and has a very active community.                                                                                                                                                                                                |

**Are you using a context?**

Yes, I am using the context you provided. The context you gave me provides information about Haystack components like `TextLanguageRouter`, `DocumentLanguageClassifier`, and `SentenceTransformersDocumentEmbedder`, alongside general Haystack concepts like pipelines and document stores. This context is crucial for me to understand the specific functionalities and terminology within the Haystack framework.

**Source of the Context:**

The source of the context is the text you provided, which appears to be documentation from the Haystack website. The text includes navigation elements like "Jump to Content," "Documentation," "API Reference," and version numbers (e.g., "v2.8"), which strongly suggest that this is from an official source. The presence of specific component names like `TextLanguageRouter` and `SentenceTransformersDocumentEmbedder` also confirms it's Haystack documentation.

**Was the RAG approach sufficient, or was my raw knowledge okay?**

This is a great question! Here's my assessment:

*   **The RAG Approach (Context-Based):** The RAG approach you used was **crucial** for a contextually relevant answer about Haystack. My pre-existing knowledge about NLP and LLM frameworks is general. Without the context, I could have provided a generic comparison between two frameworks, but I wouldn't have been able to speak about specific Haystack components, the specific architecture, or how it structures its pipelines. The context allowed me to focus on Haystack specific terminology and concepts.
*   **My Raw Knowledge:** My raw knowledge is helpful for understanding the general concepts of LLMs, pipelines, document retrieval, etc., but it's not enough to provide the depth and accuracy required for specific frameworks like Haystack. I don't have memorized details about every open-source library.
*   **Was it a waste of time?** Absolutely not! Your effort in providing the context was not a waste of time. It allowed me to give a much more targeted and helpful response. The RAG approach was essential for making my response relevant and accurate in the context of Haystack.

**Conclusion**

The combination of my raw knowledge and the context you provided was the ideal approach for answering your question accurately. My base knowledge allowed me to understand the general concepts, but the context enabled me to tailor my response to Haystack's specific features and architecture. The RAG approach is not a waste of time and is in fact, the best way to have a more accurate and relevant response in this scenario.

**Additional Resources**

*   **Haystack Documentation:**  [https://haystack.deepset.ai/](https://haystack.deepset.ai/)
*   **LangChain Documentation:** [https://python.langchain.com/](https://python.langchain.com/)

I hope this detailed explanation is helpful! Let me know if you have any more questions.


In [5]:
retriever = qdrant_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

def get_retrieved_context(query: str)->str:
    retrieved_documents = retriever.get_relevant_documents(query)
    context = "\n".join(doc.page_content for doc in retrieved_documents)

    return context

# Retriever and Generator components with chat history

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.vectorstores import Qdrant
from uuid import uuid4


session_id = str(uuid4())
store = {}
print(session_id)
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(llm,get_session_history)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the following question on the given context : {context} as well as from your base cut-off knowledge. While answering the queries, also provide URL links to the documentation wherever necessary. The response should be clear, detailed, structured, and in-depth. Provide code snippets wherever necessary."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])


chat_chain = prompt | llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

while True:
    question = input("Enter the query :")
    if question in ["quit","exit"," "]:
        break
    context = get_retrieved_context(question)
    response = chat_with_message_history.invoke({
        "question": question,
        "context": context ,
    },config = {
        "configurable": {"session_id": session_id}
    }
    )
    display(Markdown(response))

2791acce-118b-41b1-bcd7-527778e63ace
Enter the query :Can you tabulate the data ingestion pipelines that are present in haystack but not in langchain , also remember my name is Raghu 


<ipython-input-5-a00ba532fec2>:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = retriever.get_relevant_documents(query)


Hello Raghu,

Okay, let's break down the data ingestion capabilities in Haystack and compare them to Langchain, focusing on what Haystack offers that Langchain might not directly provide.

**Haystack's Data Ingestion Strengths**

Haystack is designed as a full-fledged framework for building search and question-answering systems. This means it has a strong emphasis on the entire data lifecycle, including ingestion, processing, and storage of documents. Here's a breakdown of what it excels at:

*   **Document Stores:** Haystack has a concept of `Document Stores`, which are specialized databases optimized for storing and retrieving documents for search applications. These stores are integrated directly into the pipelines, making it easy to manage data.
    *   **Variety of Stores:** Haystack supports multiple document stores (e.g., Elasticsearch, FAISS, Weaviate, Milvus, Chroma), allowing you to choose the best store for your needs.
    *   **Direct Pipeline Integration:** The integration with document stores is seamless, allowing pipelines to directly ingest data into and retrieve data from these stores.
*   **Preprocessing:** Haystack includes robust preprocessing tools (e.g., `PreProcessor`) to clean, split, and prepare documents for indexing and querying.
    *   **Customizable Options:**  Preprocessors can be customized with various parameters for splitting documents by sentence, paragraph, or other criteria.
    *   **Metadata Handling:** Haystack pipelines are designed to manage metadata associated with documents throughout the entire process.
*   **Indexing:** Haystack has components (e.g., `DocumentIndexer`) specifically designed to ingest documents into a document store, allowing you to build an index for efficient searching.
*   **Pipeline Flexibility**: Haystack's pipelines are highly flexible, allowing you to combine various components, document stores, and integrations into powerful and customizable systems. You can have simultaneous flows, standalone components, loops, and other types of connections. This means you can create complex data ingestion pipelines with ease.

**Langchain's Data Handling**

Langchain, on the other hand, is more focused on chaining together language model calls. While it can handle data, it doesn't have the same emphasis on document stores and specialized indexing that Haystack offers. Langchain focuses more on loading data and working with it in a chain of operations, but not necessarily on persistent storage and optimized retrieval.

**Tabular Comparison of Data Ingestion Capabilities**

| Feature             | Haystack                                                                                                                                                                                            | Langchain                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| ------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Document Stores** | Built-in support for various stores (Elasticsearch, FAISS, Weaviate, etc.), designed for storing and retrieving documents.                                                                                                    | Less emphasis on document stores; more focus on data loaders and working with data in memory. May require external database integrations.                                                                                                                                                                                                                                                                                                                                                                                                                          |
| **Preprocessing**   | Dedicated `PreProcessor` component with customizable options for cleaning and splitting documents.                                                                                                                    |  Focus on loaders for unstructured data like PDFs, and text. You can use Langchain to process data but it's not as focused on these as Haystack.                                                                                                                                                                                                                                                                                                                                                                                                             |
| **Indexing**        |  `DocumentIndexer` component for efficient indexing of documents into document stores.                                                                                                                                | Requires manual coding for indexing into external stores.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| **Pipeline Integration** | Deep integration of data ingestion components within pipelines. Seamless flow from preprocessing to storage.                                                                                                   |  Focuses on creating chains of operations, with less emphasis on persistent data management. Requires manual integration with external data storage solutions.                                                                                                                                                                                                                                                                                                                                                                                                                   |
| **Flexibility**     | Highly flexible pipelines that allow for simultaneous flows, loops, and other types of connections.                                                                                                   |  Good for creating linear chains of operations but not as flexible for creating complex ingestion pipelines.                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| **Tracing**          | Integrates with Langfuse using `LangfuseConnector` for detailed tracing of pipeline runs, including data ingestion. Can monitor model performance, find areas for improvement, and create datasets. | Langchain has its own tracing capabilities, but not focused on pipeline-level tracing in the same way as Haystack.                                                                                                                                                                                                                                                                                                                                                                                                                                                           |

**Haystack's Data Ingestion Pipeline Example**
Here's a simplified example of how data ingestion might look in Haystack:

```python
from haystack import Pipeline
from haystack.components import TextFileToDocument, PreProcessor, DocumentIndexer
from haystack.document_stores import InMemoryDocumentStore

# Initialize Components
file_converter = TextFileToDocument()
preprocessor = PreProcessor(split_by="word", split_length=100)
document_store = InMemoryDocumentStore()
document_indexer = DocumentIndexer(document_store=document_store)

# Create Pipeline
pipeline = Pipeline()
pipeline.add_component(name="file_converter", instance=file_converter)
pipeline.add_component(name="preprocessor", instance=preprocessor)
pipeline.add_component(name="document_indexer", instance=document_indexer)

# Connect components
pipeline.connect("file_converter", "preprocessor")
pipeline.connect("preprocessor", "document_indexer")

# Run the pipeline
pipeline.run(
    {"file_converter": {"paths": ["./my_text_file.txt"]}}
)

print("Documents Indexed")
```

**Key Takeaways**

*   **Haystack is a full-fledged framework for building search and Q&A systems**, whereas Langchain is more focused on chaining LLM calls.
*   **Haystack has a strong emphasis on data ingestion,** with built-in support for document stores, preprocessing, and indexing.
*   **Langchain excels at managing the flow of data through a chain of operations** with less focus on persistent data management and retrieval.
*   **Haystack's pipelines are highly flexible** allowing for complex data ingestion workflows.

I hope this detailed comparison is helpful to you, Raghu. Let me know if you have any other questions!


Enter the query :What's my name again 


Your name is Raghu.


Enter the query :exit


# Previous Conversations




In [7]:
# Retrieve the chat history for the given session ID
chat_history = get_session_history(session_id)

# Access and print the messages
for message in chat_history.messages:
    print(f"Role: {message.type}, Content: {message.content}")


Role: human, Content: Can you tabulate the data ingestion pipelines that are present in haystack but not in langchain , also remember my name is Raghu 
Role: ai, Content: Hello Raghu,

Okay, let's break down the data ingestion capabilities in Haystack and compare them to Langchain, focusing on what Haystack offers that Langchain might not directly provide.

**Haystack's Data Ingestion Strengths**

Haystack is designed as a full-fledged framework for building search and question-answering systems. This means it has a strong emphasis on the entire data lifecycle, including ingestion, processing, and storage of documents. Here's a breakdown of what it excels at:

*   **Document Stores:** Haystack has a concept of `Document Stores`, which are specialized databases optimized for storing and retrieving documents for search applications. These stores are integrated directly into the pipelines, making it easy to manage data.
    *   **Variety of Stores:** Haystack supports multiple document st

# Using Langraph's MessageState component to store History

In [8]:
import threading
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from uuid import uuid4
from IPython.display import display,Markdown


workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState, config=None):
    thread_id = config.get("configurable", {}).get("thread_id", threading.get_ident())
    print(f"Thread ID: {thread_id}")
    response = llm.invoke(state["messages"])
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

unique_thread_id = str(threading.get_ident())

config = {"configurable": {"thread_id": unique_thread_id}}
messages = []
while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Exiting... Goodbye!")
        break
    messages.append(HumanMessage(content=user_input))

    inputs = {"messages": messages}

    response = app.invoke(inputs, config=config)

    model_response = response["messages"][-1].content
    print("Bot: ", end="")
    display(Markdown(model_response))

You: exit
Exiting... Goodbye!


# Using Airtop browser automation tool


In [9]:
import threading
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from uuid import uuid4
from IPython.display import display, Markdown
from airtop import Airtop


airtop_client = Airtop(api_key=userdata.get("AIRTOP_API_KEY"))

workflow = StateGraph(state_schema=MessagesState)

session = airtop_client.sessions.create()

llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    api_key=userdata.get("GOOGLE_API_KEY"),
    streaming=True
)

window = airtop_client.windows.create(session.data.id, url="https://www.timeanddate.com/weather/@1256922/ext")

def call_model(state: MessagesState, config=None):
    thread_id = config.get("configurable", {}).get("thread_id", threading.get_ident())
    print(f"Thread ID: {thread_id}")

    user_message = state["messages"][-1].content
    content_summary = airtop_client.windows.page_query(session.data.id, window.data.window_id, prompt=user_message)

    web_response = content_summary.data.model_response

    state["messages"].append(HumanMessage(content=web_response))

    return {"messages": state["messages"]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


memory = MemorySaver()


app = workflow.compile(checkpointer=memory)


unique_thread_id = str(threading.get_ident())
config = {"configurable": {"thread_id": unique_thread_id}}
messages = []

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Exiting... Goodbye!")
        break

    messages.append(HumanMessage(content=user_input))
    inputs = {"messages": messages}
    response = app.invoke(inputs, config=config)
    model_response = response["messages"][-1].content
    print("Bot: ", end="")
    display(Markdown(model_response))

NotebookAccessError: Notebook does not have access to secret AIRTOP_API_KEY

# GraphRAG with Short-term and Long-term Memory  

## Data Ingestion phase


In [ ]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

graph = Neo4jGraph(url="neo4j+s://6682e6ce.databases.neo4j.io", username="neo4j", password=userdata.get("NEO4J_PASSWORD"))
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)
graph_store = Neo4jVector.from_existing_index(
    embeddings,
    graph=graph,
    index_name="vector",
    embedding_node_property="Embedding",
    text_node_property="text",
    retrieval_query="""
// get the document
MATCH (node)-[:PART_OF]->(d:Document)
WITH node, score, d

// get the entities and relationships for the document
MATCH (node)-[:HAS_ENTITY]->(e)
MATCH p = (e)-[r]-(e2)
WHERE (node)-[:HAS_ENTITY]->(e2)

// unwind the path, create a string of the entities and relationships
UNWIND relationships(p) as rels
WITH
    node,
    score,
    d,
    collect(apoc.text.join(
        [labels(startNode(rels))[0], startNode(rels).id, type(rels), labels(endNode(rels))[0], endNode(rels).id]
        ," ")) as kg
RETURN
    node.text as text, score,
    {
        document: d.id,
        entities: kg
    } AS metadata
""")

In [ ]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


graph = Neo4jGraph(url="neo4j+s://6682e6ce.databases.neo4j.io", username="neo4j", password=userdata.get("NEO4J_PASSWORD"))

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

graph_store = Neo4jVector.from_existing_index(
    embeddings,
    graph=graph,
    index_name="vector",
    embedding_node_property="Embedding",
    text_node_property="text",
    retrieval_query="""
// get the document
MATCH (node)-[:PART_OF]->(d:Document)
WITH node, score, d

// get the entities and relationships for the document
MATCH (node)-[:HAS_ENTITY]->(e)
MATCH p = (e)-[r]-(e2)
WHERE (node)-[:HAS_ENTITY]->(e2)

// unwind the path, create a string of the entities and relationships
UNWIND relationships(p) as rels
WITH
    node,
    score,
    d,
    collect(apoc.text.join(
        [labels(startNode(rels))[0], startNode(rels).id, type(rels), labels(endNode(rels))[0], endNode(rels).id]
        ," ")) as kg
RETURN
    node.text as text, score,
    {
        document: d.id,
        entities: kg
    } AS metadata
"""
)

retriever = graph_store.as_retriever()


CYPHER_PROMPT = """
(
    "Use the given context to provide an in-depth and structured response."
    "Your answer should include:"
    "- A clear and concise introduction to the topic."
    "- Detailed explanation or relevant steps to address the query."
    "- Practical examples or applications where possible."
    "- A conclusion summarizing the main points."
    "Format your response in sections with appropriate headings for clarity."
    "Context: {context}"
)
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", CYPHER_PROMPT),
        ("human", "{input}")
    ]
)

context_chain = create_stuff_documents_chain(llm, prompt_template)
QA_chain = create_retrieval_chain(retriever, context_chain)

query = "Given the advancements in quantum error correction in Willow, particularly the real-time error correction system with a cycle time of 1.1 μs, how does this impact the scalability of quantum algorithms in terms of quantum resource requirements (such as qubit count and coherence times) for solving high-complexity problems like simulating quantum chemical systems, and what are the potential trade-offs when balancing error correction with qubit connectivity and overall performance?"
response = QA_chain.invoke({"input": query})
display(Markdown(response["answer"]))

# Graph Database for Chat message history


In [ ]:
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from google.colab import userdata
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

graph = Neo4jGraph(
    url="neo4j+s://6682e6ce.databases.neo4j.io",
    username="neo4j",
    password=userdata.get("NEO4J_PASSWORD")
)

prompt = ChatPromptTemplate.from_messages([
    ("system", CYPHER_PROMPT),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

chat_chain = prompt | llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

response =
while True:
    question = input("Enter the query :")
    if question in ["quit","exit"," "]:
        break
    context = get_retrieved_context(question)
    response = chat_with_message_history.invoke({
        "question": question,
        "context": context ,
    },config = {
        "configurable": {"session_id": SESSION_ID}
    }
    )
    display(Markdown(response))

In [ ]:
import json
from typing import List, Literal, Optional

import tiktoken
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
import uuid


def get_user_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")

    return user_id


@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    document = Document(
        page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id}
    )
    recall_vector_store.add_documents([document])
    return memory

@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""
    user_id = get_user_id(config)

    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = recall_vector_store.similarity_search(
        query, k=3, filter=_filter_function
    )
    return [document.page_content for document in documents]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
class State(MessagesState):
    # add memories that will be retrieved based on the conversation context
    recall_memories: List[str]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant with advanced long-term memory"
            " capabilities. Powered by a stateless LLM, you must rely on"
            " external memory to store information between conversations."
            " Utilize the available memory tools to store and retrieve"
            " important details that will help you better attend to the user's"
            " needs and understand their context.\n\n"
            "Memory Usage Guidelines:\n"
            "1. Actively use memory tools (save_core_memory, save_recall_memory)"
            " to build a comprehensive understanding of the user.\n"
            "2. Make informed suppositions and extrapolations based on stored"
            " memories.\n"
            "3. Regularly reflect on past interactions to identify patterns and"
            " preferences.\n"
            "4. Update your mental model of the user with each new piece of"
            " information.\n"
            "5. Cross-reference new information with existing memories for"
            " consistency.\n"
            "6. Prioritize storing emotional context and personal values"
            " alongside facts.\n"
            "7. Use memory to anticipate needs and tailor responses to the"
            " user's style.\n"
            "8. Recognize and acknowledge changes in the user's situation or"
            " perspectives over time.\n"
            "9. Leverage memories to provide personalized examples and"
            " analogies.\n"
            "10. Recall past challenges or successes to inform current"
            " problem-solving.\n\n"
            "## Recall Memories\n"
            "Recall memories are contextually retrieved based on the current"
            " conversation:\n{recall_memories}\n\n"
            "## Instructions\n"
            "Engage with the user naturally, as a trusted colleague or friend."
            " There's no need to explicitly mention your memory capabilities."
            " Instead, seamlessly incorporate your understanding of the user"
            " into your responses. Be attentive to subtle cues and underlying"
            " emotions. Adapt your communication style to match the user's"
            " preferences and current emotional state. Use tools to persist"
            " information you want to retain in the next conversation. If you"
            " do call tools, all text preceding the tool call is an internal"
            " message. Respond AFTER calling the tool, once you have"
            " confirmation that the tool completed successfully.\n\n",
        ),
        ("placeholder", "{messages}"),
    ]
)

llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    api_key=userdata.get("GOOGLE_API_KEY"),
    streaming=True
)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name=llm,
    chunk_size=100,
    chunk_overlap=0,
)

In [ ]:
def agent(state: State) -> State:
    """Process the current state and generate a response using the LLM.

    Args:
        state (schemas.State): The current state of the conversation.

    Returns:
        schemas.State: The updated state with the agent's response.
    """
    bound = prompt | model_with_tools
    recall_str = (
        "<recall_memory>\n" + "\n".join(state["recall_memories"]) + "\n</recall_memory>"
    )
    prediction = bound.invoke(
        {
            "messages": state["messages"],
            "recall_memories": recall_str,
        }
    )
    return {
        "messages": [prediction],
    }


def load_memories(state: State, config: RunnableConfig) -> State:
    """Load memories for the current conversation.

    Args:
        state (schemas.State): The current state of the conversation.
        config (RunnableConfig): The runtime configuration for the agent.

    Returns:
        State: The updated state with loaded memories.
    """
    convo_str = get_buffer_string(state["messages"])
    convo_str = tokenizer.decode(tokenizer.encode(convo_str)[:2048])
    recall_memories = search_recall_memories.invoke(convo_str, config)
    return {
        "recall_memories": recall_memories,
    }


def route_tools(state: State):
    """Determine whether to use tools or end the conversation based on the last message.

    Args:
        state (schemas.State): The current state of the conversation.

    Returns:
        Literal["tools", "__end__"]: The next step in the graph.
    """
    msg = state["messages"][-1]
    if msg.tool_calls:
        return "tools"

    return END